In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from keras import backend as K
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPool2D, Flatten, Dense,TimeDistributed
from keras.models import Model, load_model
from keras.layers.convolutional import ZeroPadding2D
from keras import metrics
import h5py
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VIDEOS_DIR = '../Videos/'
IMAGES_DIR = '../Images/'
classes = []
class_to_index = {}
videos = []

classes = ['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']


for i in range(len(classes)):
    class_to_index[classes[i]] = i
class_to_index

for x in classes:
    videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
videos

[['006',
  '017',
  '003',
  '016',
  '009',
  '013',
  '005',
  '012',
  '010',
  '015',
  '014',
  '007',
  '004',
  '011',
  '002',
  '001'],
 ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'],
 ['006', '009', '005', '010', '007', '004', '002', '008', '001'],
 ['006', '003', '009', '005', '010', '007', '004', '002', '008', '001'],
 ['006',
  '017',
  '003',
  '016',
  '009',
  '013',
  '005',
  '012',
  '010',
  '015',
  '014',
  '007',
  '004',
  '011',
  '002',
  '008',
  '001'],
 ['003', '005', '004', '002', '001'],
 ['006', '003', '009', '005', '010', '007', '004', '011', '002', '008', '001'],
 ['006',
  '017',
  '003',
  '016',
  '009',
  '013',
  '005',
  '012',
  '010',
  '015',
  '014',
  '007',
  '019',
  '004',
  '011',
  '018',
  '002',
  '008',
  '001'],
 ['006',
  '003',
  '009',
  '013',
  '005',
  '012',
  '010',
  '014',
  '007',
  '004',
  '011',
  '002',
  '008',
  '001']]

## Function Block

In [3]:
def permute(X,Y):
    train_size = X.shape[0]
    permutation_train = np.random.permutation(train_size)
    X = X[permutation_train]
    Y = Y[permutation_train]
    return X,Y

def load_image(path,image_size):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    return image

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def pad(X_train_images_class,max_len):
    length = len(X_train_images_class)
    pad_arr = np.zeros((X_train_images_class.shape[1:4]),dtype=np.uint8)
    X_train_images_class = list(X_train_images_class)
    for i in range(max_len-length):
        X_train_images_class.append(pad_arr)
    return np.array(X_train_images_class,dtype=np.uint8)

def evaluate(model, X_test,Y_test,verbose = True):
    count = 0
    for i in range(len(X_test)):
        pred = model.predict(X_test[i])[0]
        #print(pred[0].shape, pred[1].shape)
        #break
        max_pred = [np.argmax(i) for i in pred]
        counts = np.bincount(max_pred)
        class_pred = np.argmax(counts)
        #class_pred = max_pred
        #actual = np.argmax(Y_test[i])
        actual = Y_test[i]
        if verbose:
            print("Max Preds time", max_pred)
            print("Pred",classes[class_pred],"Actual",classes[actual])
            print()
        if class_pred == actual:
            count += 1
    return float(count)/float(len(Y_test)) * 100.0

In [21]:
def build_dataset_end_to_end(image_size, max_len = 40, stride = 10):
    
    X_train_images = []
    Y_train_images = []
    
    X_test_images = []
    Y_test_images = []
    
    test_videos = [['004', '011', '007'], ['006', '011'], ['007', '003'], \
                   ['003','001'], ['006', '012', '009'], ['004', '005'], ['008','002'], ['004', '012', '002'], ['001', '013', '006']]
    
    for i in range(len(classes)):
        cls = classes[i]
        test = test_videos[i] 
        for j in range(len(videos[i])):
            vid = videos[i][j]
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            filelist = sorted(list(os.listdir(image_r)))
            X_train_images_class = []
            for file in filelist:
                if file.endswith(".png"):
                    image = load_image(image_r+file,image_size)
                    X_train_images_class.append(image)
            X_train_images_class = np.array(X_train_images_class)        
            X_test_frames = []                                
            for k in range(0,len(X_train_images_class),stride):
                lower = k
                upper = min(len(X_train_images_class),k+max_len)
                if upper == len(X_train_images_class):
                    if vid not in test:                
                        X_train_images.append(pad(X_train_images_class[lower:upper],max_len))
                        Y_train_images.append(i)
                    else:
                        X_test_frames.append(pad(X_train_images_class[lower:upper],max_len))
                        X_test_images.append(np.array(X_test_frames))        
                        Y_test_images.append(i)
                    print("Padded frames" , lower , "to" , upper)
                    break
                else:
                    if vid not in test:                
                        X_train_images.append(X_train_images_class[lower:upper])
                        Y_train_images.append(i)
                    else:
                        X_test_frames.append(X_train_images_class[lower:upper])
                    print("Added frames" , lower , "to" , upper)
                    
            print("Processed",videos[i][j],"of","class",classes[i])
    return np.array(X_train_images,dtype=np.uint8),np.array(Y_train_images,dtype=np.uint8), np.array(X_test_images), np.array(Y_test_images)

In [22]:
X_train = None
X_test = None
Y_train = None
Y_test = None
try:
    X_train = np.load('../Numpy/End2End/X_train.npy')
    Y_train = np.load('../Numpy/End2End/Y_train.npy')
    X_test = np.load('../Numpy/End2End/X_test.npy')
    Y_test = np.load('../Numpy/End2End/Y_test.npy')
except FileNotFoundError:
    X_train, Y_train, X_test, Y_test = build_dataset_end_to_end((172, 172))
    np.save('../Numpy/End2End/X_train.npy', X_train)
    np.save('../Numpy/End2End/Y_train.npy', Y_train)
    np.save('../Numpy/End2End/X_test.npy', X_test)
    np.save('../Numpy/End2End/Y_test.npy', Y_test)

In [26]:
X_train.shape, X_test.shape,Y_train.shape, Y_test.shape

((345, 40, 172, 172, 3), (20,), (345,), (20,))

In [7]:
def end_to_end(input_shape):
    X_input = Input(input_shape)
    X = TimeDistributed(BatchNormalization(name = 'BatchNorm_1'))(X_input)
    #X = TimeDistributed(ZeroPadding2D((3, 3)))(X)
    X = TimeDistributed(Conv2D(32, (7, 7), strides = (4, 4), activation='relu', name="Conv_1a", padding="same"))(X)
    X = TimeDistributed(Conv2D(32, (3, 3), activation='relu', name="Conv_1b", padding="same"))(X)
    X = TimeDistributed(MaxPool2D((2, 2), name = "Pool_1"))(X)
    X = TimeDistributed(Dropout(0.2))(X)
    
    X = TimeDistributed(Conv2D(32, (3, 3), name ="Conv_2a", activation='relu', padding = "same"))(X)
    #X = TimeDistributed(Conv2D(32, (3, 3), name ="Conv_2b", activation='relu', padding = "same"))(X)
    X = TimeDistributed(MaxPool2D((2, 2), name = "Pool_2"))(X)
    X = TimeDistributed(Dropout(0.2))(X)
    X = TimeDistributed(Conv2D(32,(3,3), name='Conv_3a'))(X)
    X = TimeDistributed(MaxPool2D((2, 2), name = "Pool_3"))(X)
    
    X = TimeDistributed(Conv2D(8,(1,1), name='Conv_1x1'))(X)
    X = TimeDistributed(Flatten())(X)
    X = TimeDistributed(Dropout(0.3))(X)
    Y = TimeDistributed(Dense(9,activation='softmax',name='final'))(X)
    
    #X = LSTM(32, return_sequences=True)(X)
    X = LSTM(32, return_sequences=False)(X)
    X = Dense(9, activation='softmax')(X)

    return Model(X_input, outputs=[X, Y])

In [8]:
e2e = end_to_end((40, 172, 172, 3))
print(e2e.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 172, 172, 0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 40, 172, 172, 12          input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 40, 43, 43, 3 4736        time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 40, 43, 43, 3 9248        time_distributed_2[0][0]         
__________________________________________________________________________________________________
time_distr

In [9]:
e2e.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.5],
        metrics=['accuracy'],
        optimizer='adam')

In [10]:
X_train, Y_train = permute(X_train, Y_train)

In [11]:
X_train.shape

(375, 40, 172, 172, 3)

In [12]:
Y_train = convert_to_one_hot(Y_train, 9)


In [13]:
Y_train.shape

(375, 9)

In [14]:
Y_train2 = np.tile(Y_train, (40, 1, 1))

In [15]:
Y_train2 = Y_train2.transpose(1, 0, 2)


In [16]:
Y_train2.shape

(375, 40, 9)

## Training

In [28]:
for i in range(0,10):
    file='temp' + str(i) + '.h5' 
    mod = load_model('temp' + str(i) + '.h5')
    result = evaluate(mod,X_test,Y_test,verbose=False)
    print(file,result)

temp0.h5 85.0
temp1.h5 85.0
temp2.h5 85.0
temp3.h5 70.0
temp4.h5 80.0
temp5.h5 80.0
temp6.h5 70.0
temp7.h5 80.0
temp8.h5 85.0
temp9.h5 75.0


In [29]:
for i in range(5):
    mod_16.fit(X_train, [Y_train, Y_train2], epochs = 1, batch_size = 64, validation_split = 0.2)
    mod_16.save('temp' + str(i+5) + '.h5')
    result = evaluate(mod_16,X_test,Y_test,verbose=False)
    print(result)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-5a5bb0d55b4e>", line 1, in <module>
    mod_16 = load_model('temp0.h5')
  File "/usr/local/lib/python3.5/dist-packages/keras/models.py", line 246, in load_model
    topology.load_weights_from_hdf5_group(f['model_weights'], model.layers)
  File "/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py", line 3382, in load_weights_from_hdf5_group
    K.batch_set_value(weight_value_tuples)
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 2373, in batch_set_value
    get_session().run(assign_ops, feed_dict=feed_dict)
  File "/usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py", line 199, in get_session
    session.run(tf.variables_initializer(uninitialized_vars))
  File "/usr/local/lib/python3.5/dist-pac

KeyboardInterrupt: 

In [30]:
X_train = np.load('../Numpy/End2End/X_train_10_40.npy')
X_test = np.load('../Numpy/End2End/X_test_10_40.npy')
Y_test = np.load('../Numpy/End2End/Y_test_10_40.npy')
Y_train = np.load('../Numpy/End2End/Y_train_10_40.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../Numpy/End2End/X_train_10_40.npy'

In [ ]:
X_train.shape, Y_train_shape

## Testing

In [48]:
def build_test_dataset(image_size, stride = 10, max_len = 40):
    
    X_test_images = []
    Y_test_images = []
    
    VIDEOS_DIR = '../Videos/'
    IMAGES_DIR = '../Images/'
    classes = ['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']
    videos = []
    for x in classes:
        videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
    
    for i in range(len(classes)):
        cls = classes[i]
        
        for j in range(len(videos[i])):
            vid = videos[i][j]
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            filelist = sorted(list(os.listdir(image_r)))
            X_train_images_class = []
            for file in filelist:
                if file.endswith(".png"):
                    image = load_image(image_r+file,image_size)
                    X_train_images_class.append(image)
            X_train_images_class = np.array(X_train_images_class)        
            X_test_frames = []                                
            for k in range(0,len(X_train_images_class),stride):
                lower = k
                upper = min(len(X_train_images_class),k+max_len)
                if upper == len(X_train_images_class):             
                    X_test_frames.append(pad(X_train_images_class[lower:upper],max_len))
                    X_test_images.append(np.array(X_test_frames))        
                    Y_test_images.append(i)
                    print("Padded frames" , lower , "to" , upper)
                    break
                else:
                    X_test_frames.append(X_train_images_class[lower:upper])
                    print("Added frames" , lower , "to" , upper)
                    
            print("Processed",videos[i][j],"of","class",classes[i])
    return np.array(X_test_images), np.array(Y_test_images)

In [49]:
X_test_unseen = None
Y_test_unseen = None
try:
    X_test_unseen = np.load('../Numpy/End2End/X_test_full_training.npy')
    Y_test_unseen = np.load('../Numpy/End2End/Y_test_full_training.npy')
except FileNotFoundError:
    X_test_unseen, Y_test_unseen = build_test_dataset((172,172))
    np.save('../Numpy/End2End/X_test_full_training.npy', X_test_unseen)
    np.save('../Numpy/End2End/Y_test_full_training.npy', Y_test_unseen)

Padded frames 0 to 23
Processed 006 of class Kicking
Padded frames 0 to 23
Processed 017 of class Kicking
Padded frames 0 to 23
Processed 003 of class Kicking
Padded frames 0 to 23
Processed 016 of class Kicking
Padded frames 0 to 22
Processed 009 of class Kicking
Padded frames 0 to 23
Processed 013 of class Kicking
Padded frames 0 to 23
Processed 005 of class Kicking
Padded frames 0 to 23
Processed 012 of class Kicking
Padded frames 0 to 23
Processed 010 of class Kicking
Padded frames 0 to 23
Processed 015 of class Kicking
Padded frames 0 to 23
Processed 014 of class Kicking
Padded frames 0 to 23
Processed 007 of class Kicking
Padded frames 0 to 23
Processed 004 of class Kicking
Padded frames 0 to 23
Processed 011 of class Kicking
Padded frames 0 to 23
Processed 002 of class Kicking
Padded frames 0 to 23
Processed 001 of class Kicking
Padded frames 0 to 39
Processed 006 of class Riding-Horse
Added frames 0 to 40
Added frames 10 to 50
Padded frames 20 to 60
Processed 003 of class Ridin

Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Added frames 30 to 70
Added frames 40 to 80
Added frames 50 to 90
Added frames 60 to 100
Padded frames 70 to 101
Processed 003 of class Walking
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Added frames 30 to 70
Added frames 40 to 80
Added frames 50 to 90
Added frames 60 to 100
Added frames 70 to 110
Added frames 80 to 120
Added frames 90 to 130
Added frames 100 to 140
Padded frames 110 to 144
Processed 016 of class Walking
Padded frames 0 to 37
Processed 009 of class Walking
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Added frames 30 to 70
Added frames 40 to 80
Added frames 50 to 90
Added frames 60 to 100
Padded frames 70 to 101
Processed 013 of class Walking
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Added frames 30 to 70
Added frames 40 to 80
Added frames 50 to 90
Added frames 60 to 100
Padded frames 70 to 101
Processed 005 of class Walking
Added frames 0 to

In [50]:
Y_test_unseen = convert_to_one_hot(np.array(Y_test_unseen), 9)
Y_test_unseen.shape

(111, 9)

In [51]:
[test.shape for test in X_test_unseen]

[(1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (1, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (3, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 172, 172, 3),
 (4, 40, 17

In [37]:
def evaluate(model, X_test,Y_test,verbose = True):
    count = 0
    for i in range(len(X_test)):
        pred = model.predict(X_test[i])[0]
        max_pred = [np.argmax(i) for i in pred]
        counts = np.bincount(max_pred)
        class_pred = np.argmax(counts)
        actual = np.argmax(Y_test[i])
        if verbose:
            print("Max Preds time", max_pred)
            print("Pred",classes[class_pred],"Actual",classes[actual])
            print()
        if class_pred == actual:
            count += 1
    return float(count)/float(len(Y_test)) * 100.0

In [55]:
for i in os.listdir('../models/End_End/'):
    if i[-3:] == '.h5':
        mod_16 = load_model('../models/End_End/' + i)
        res = evaluate(mod_16, X_test_unseen, Y_test_unseen,verbose=False)
        print(i,res)

temp9.h5 91.8918918918919
temp8.h5 94.5945945945946
temp5.h5 94.5945945945946
temp7.h5 95.4954954954955
temp0.h5 90.990990990991
temp4.h5 94.5945945945946


In [ ]:
np.save('../Numpy/Unseen/Y_test_Unseen_10_40', Y_test)
